<a href="https://colab.research.google.com/github/adityanation/rag-chatbot-vercel/blob/main/RAG_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Required Libraries

In [1]:
!pip install requests beautifulsoup4 faiss-cpu sentence-transformers transformers gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.

Scrape Data from a Website

In [11]:
import requests
from bs4 import BeautifulSoup

def scrape_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract text from <p> tags
    paragraphs = soup.find_all("p")
    text = "\n".join([p.get_text() for p in paragraphs])

    return text

# Example: Scrape Wikipedia page on Food Waste
scraped_text = scrape_text("https://en.wikipedia.org/wiki/Artificial_intelligence")  # AI topic
print(scraped_text[:500])  # Print first 500 characters




Artificial intelligence (AI) refers to the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be cal


Split Text into Chunks

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_text(text, chunk_size=500, overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    return text_splitter.split_text(text)

text_chunks = split_text(scraped_text)
print("Number of chunks:", len(text_chunks))


Number of chunks: 249


Convert Text into Embeddings

In [14]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

# Convert text into embeddings
embeddings = [model.encode(chunk) for chunk in text_chunks]
print("Embedding shape:", np.array(embeddings).shape)


Embedding shape: (249, 384)


Store & Retrieve Using FAISS

In [16]:
import faiss

# Store embeddings in FAISS
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

def retrieve_top_k(query, k=3):
    query_vector = model.encode(query).reshape(1, -1)
    distances, indices = index.search(query_vector, k)
    return [text_chunks[i] for i in indices[0]]

# Test retrieval
query = "How to reduce food waste?"
retrieved_texts = retrieve_top_k(query)
print("Retrieved:", retrieved_texts)


Retrieved: ["Several approaches aim to address the transparency problem. SHAP enables to visualise the contribution of each feature to the output.[262] LIME can locally approximate a model's outputs with a simpler, interpretable model.[263] Multitask learning provides a large number of outputs in addition to the target classification. These other outputs can help developers deduce what the network has learned.[264] Deconvolution, DeepDream and other generative methods can allow developers to see what", 'Arguments for decomputing have been raised by Dan McQuillan (Resisting AI: An Anti-fascist Approach to Artificial Intelligence, 2022), meaning an opposition to the sweeping application and expansion of artificial intelligence. Similar to degrowth the approach criticizes AI as an outgrowth of the systemic issues and capitalist world we live in. Arguing that a different future is possible, in which distance between people is reduced and not increased to AI intermediaries.[420]', 'overall 

Generate Responses Using GPT-2

In [17]:
def generate_response(query):
    retrieved_texts = retrieve_top_k(query)
    context = " ".join(retrieved_texts)
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"

    # Use max_new_tokens instead of max_length
    response = generator(prompt, max_new_tokens=50, truncation=True)

    return response[0]["generated_text"]



In [18]:
def generate_response(query):
    retrieved_texts = retrieve_top_k(query, k=2)  # Reduce retrieved chunks if needed
    context = " ".join(retrieved_texts)

    # Use truncation to limit the input length
    prompt = f"Context: {context[:500]}\nQuestion: {query[:100]}\nAnswer:"  # Limit context & query length

    # Generate response with max_new_tokens instead of max_length
    response = generator(prompt, max_new_tokens=50, truncation=True, pad_token_id=50256)

    return response[0]["generated_text"]


Create a Web App Using Gradio

In [21]:
import gradio as gr

def chatbot(query):
    return generate_response(query)

gr.Interface(fn=chatbot, inputs="text", outputs="text", title="RAG Chatbot").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7dbcb1235cfde2c1be.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
